# 라이브러리 임포트

In [3]:
import pandas as pd
import numpy as np

import FinanceDataReader as fdr
from statsmodels.tsa.seasonal import seasonal_decompose

from xgboost import XGBRegressor

import os 
from tqdm import tqdm
import warnings 
warnings.filterwarnings(action = 'ignore') 
pd.options.display.max_columns = None

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 데이터셋 로드

In [4]:
path = '../dacon stock closing price/dataset'

stock_list = pd.read_csv(path + '/Stock_List.csv')
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x: str(x).zfill(6))
stock_codes = np.sort(stock_list['종목코드'].values)

# 전처리를 위한 함수 정의

In [5]:
# 이동평균 추가하는 함수 정의 
def add_rolling_mean(df:pd.DataFrame, col_name:str, window_list:list)->pd.DataFrame:
  for window in window_list:
    df[col_name + f'_rolling{window}'] = df[col_name].rolling(window).mean().fillna(method = 'bfill') 
  return df

# 컬럼값을 lag하는 함수 정의
def lag_features(df:pd.DataFrame, col_list:list, lag_num:int)->pd.DataFrame:
  for col in col_list:
    df[col] = df[col].shift(lag_num).fillna(method = 'ffill').fillna(method = 'bfill')
  return df

# 종목별 전처리

In [6]:
stock_df_dict = {}

# 종목별로 stock_df 만들어서 딕셔너리 stock_df_dict에 저장하기 
for stock_code in tqdm(stock_codes): 

  # 날짜 설정
  dates = pd.date_range('20110101', '20210917', freq = 'B')
  df = pd.DataFrame(dates, columns = ['Date'])

  # 연, 월, 주, 일, 요일, 월별주수 정보 추가
  df['Year'] = df['Date'].dt.year
  df['Month'] = df['Date'].dt.month
  df['Week'] = df['Date'].dt.week
  df['Day'] = df['Date'].dt.day
  df['Weekday'] = df['Date'].dt.weekday
  df['WeekNum'] = np.ceil(df['Day']/7).astype(int)

  # 20110101 ~ 20210910 까지의 주가 정보 가져오기
  stock_value = fdr.DataReader(stock_code, start = '20110101', end = '20210910').reset_index()
  df = pd.merge(df, stock_value, on = 'Date', how = 'left')
  df[stock_value.columns] = df[stock_value.columns].ffill() # 결측치를 이전 값으로 대체 (공휴일 등 쉬는 날에 대하여)
  df = df.dropna() # 남아있는 결측치는 상장전이므로 drop 

  # 종가 데이터 시계열 분해
  df = df.set_index('Date') 
  decomposition = seasonal_decompose(df[:-5]['Close'], model='multiplicative') 
  df['Close_trend'] = decomposition.trend.fillna(method = 'ffill').fillna(method = 'bfill') # 종가 추세 
  df['Close_seosonal'] = decomposition.seasonal.fillna(method = 'ffill').fillna(method = 'bfill') # 종가 계절성 

  # 5일, 20일, 60일, 120일, 240일 이동평균 추가 
  df = add_rolling_mean(df, 'Close', [5, 20, 60, 120, 240]) 

  # lag features 
  col_list = list(df.iloc[:, 6:].columns)
  col_list.remove('Close')
  df = lag_features(df, col_list, 5)
  df.drop(df.index[:5])

  df = df.reset_index()
  df['Index'] = df.index 
  stock_df_dict[stock_code] = df

100%|██████████| 376/376 [05:33<00:00,  1.13it/s]


# 모델링 (XGBoost)

In [7]:
# SEED 설정 
SEED = 42

# NMAE 함수 정의
def NMAE(true, pred):
 return np.mean(abs(true - pred) / true) * 100 

## public

In [8]:
# public 
xgb_oof_score = []
xgb_pred_dict = {}

for stock_code, df in tqdm(stock_df_dict.items()):
  stock_name = str(stock_list.loc[stock_list['종목코드'] == stock_code]['종목명'].values).strip("['']")
  print(f"\n[종목코드 {stock_code} 종목명 {stock_name} 학습 및 예측]")

  # split train, valid, test (public)
  train = stock_df_dict[stock_code][:-15]
  test = stock_df_dict[stock_code][-15:-10]

  X_train = train[:-5].drop(columns = ['Date', 'Close'])
  y_train = train[:-5]['Close']
  X_valid = train[-5:].drop(columns = ['Date', 'Close'])
  y_valid = train[-5:]['Close']
  X_test = test.drop(columns = ['Date', 'Close'])  

  # xgboost 
  xgb = XGBRegressor(n_estimators= 10000, learning_rate = 0.05, tree_method = 'gpu_hist', random_state = SEED)
  xgb.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 100, eval_metric = 'mae', verbose = 100)

  xgb_oof = xgb.predict(X_valid)
  xgb_pred = xgb.predict(X_test)
  xgb_pred_dict[stock_code] = xgb_pred

  # validation NMAE score 
  nmae = NMAE(y_valid, xgb_oof)
  print(f"\n{train[-5:]['Date'].min().strftime('%Y-%m-%d')} ~ {train[-5:]['Date'].max().strftime('%Y-%m-%d')} 기간동안의 Public OOF NMAE Score : {nmae:.6f}\n")
  xgb_oof_score.append(nmae)
  print('=' * 120)

print(f'전체 종목에 대한 평균 Public OOF NMAE Score : {np.mean(xgb_oof_score)}')

  0%|          | 0/376 [00:00<?, ?it/s]


[종목코드 000060 종목명 메리츠화재 학습 및 예측]
[07:00:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:24051
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:560.16
[200]	validation_0-mae:699.741
Stopping. Best iteration:
[117]	validation_0-mae:508.808



  0%|          | 1/376 [00:00<04:58,  1.26it/s]


2021-08-23 ~ 2021-08-27 기간동안의 Public OOF NMAE Score : 1.993146


[종목코드 000080 종목명 하이트진로 학습 및 예측]
[07:00:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:30571.8
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:1248.78
[200]	validation_0-mae:1571.77
Stopping. Best iteration:
[119]	validation_0-mae:1187.68



  1%|          | 2/376 [00:01<03:52,  1.61it/s]

...


2021-08-23 ~ 2021-08-27 기간동안의 Public OOF NMAE Score : 1.583401


[종목코드 363280 종목명 티와이홀딩스 학습 및 예측]
[07:04:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:28463.6
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:538.363
[200]	validation_0-mae:414.22
Stopping. Best iteration:
[192]	validation_0-mae:388.543



100%|█████████▉| 375/376 [03:35<00:00,  2.28it/s]


2021-08-23 ~ 2021-08-27 기간동안의 Public OOF NMAE Score : 1.295329


[종목코드 950130 종목명 엑세스바이오 학습 및 예측]
[07:04:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:15682
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:4117.14
Stopping. Best iteration:
[23]	validation_0-mae:195.361



100%|██████████| 376/376 [03:36<00:00,  1.74it/s]


2021-08-23 ~ 2021-08-27 기간동안의 Public OOF NMAE Score : 1.124334

전체 종목에 대한 평균 Public OOF NMAE Score : 2.972405023477817


In [9]:
xgb_public_sub = pd.DataFrame({'Day' : pd.date_range('20210830', '20210903', freq = 'B')})

for stock_code, pred in xgb_pred_dict.items():
  stock_pred = pd.DataFrame({stock_code : pred})
  xgb_public_sub = pd.concat([xgb_public_sub, stock_pred], axis = 1)

xgb_public_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,005490,005830,005850,005930,005935,005940,006260,006280,006360,006400,006650,006730,006800,007070,007310,007390,008560,008770,008930,009150,009240,009540,009830,009900,010060,010120,010130,010620,010780,010950,011000,011070,011170,011200,011210,011780,012330,012450,012510,012750,013120,013890,014680,015750,015760,016360,016380,017670,017800,018260,018880,019170,020150,021240,022100,023530,024110,025900,025980,026960,027360,028050,028260,028300,028670,029780,029960,030000,030190,030200,030520,030530,031390,031430,032190,032300,032350,032500,032620,032640,032830,033290,033310,033640,033780,034020,034220,034230,034730,035080,035250,035420,035600,035720,035760,035890,035900,036030,036460,036490,036540,036570,036810,036830,036930,038500,039030,039200,039490,039840,041190,041510,041960,042000,042660,042700,043150,045390,046890,047040,047050,047810,048260,048410,048530,049070,051600,051900,051905,051910,051915,052400,052690,053030,053580,053800,055550,056190,058470,058820,060150,060250,060720,061970,064260,064350,064550,064760,065660,066570,066575,066970,067000,067160,067310,067630,068240,068270,068760,069080,069500,069620,069960,071050,073240,074600,078020,078130,078340,078600,078930,081660,082270,083790,084110,084370,084850,084990,085660,086280,086390,086450,086790,086900,088350,088800,089030,089970,089980,090430,090460,091700,091990,092040,092190,095610,095660,095700,096530,096770,097950,098460,099320,100090,101490,102110,102710,102780,104480,105560,108320,110790,111770,112040,112610,120110,121800,122450,122630,122870,128940,131290,131370,131970,137400,138040,138930,139130,139480,140410,140860,141080,143240,144510,145020,151860,151910,153130,161390,161890,166090,175330,178320,178920,180640,181710,183300,183490,185490,185750,192820,194480,196170,200130,204270,204320,205470,206640,206650,207940,213420,214150,214320,214370,214450,214980,215000,215200,216080,217270,218410,220630,222080,222800,226950,230360,232140,235980,237690,240810,241560,243070,243840,247540,251270,252670,253450,256840,263720,263750,267250,267980,268600,271560,272210,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,298540,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-08-30,24925.125000,31336.667969,61190.394531,167265.375000,88352.039062,17018.898438,54904.292969,81078.289062,103442.5625,650774.5625,48399.035156,221944.562500,31986.554688,55878.007812,17921.396484,2374.069824,25877.228516,5391.016602,311403.09375,56400.218750,21530.740234,34567.867188,9902.660156,7983.778320,29158.724609,147513.406250,47799.410156,64871.878906,46388.074219,250994.531250,295004.71875,81688.648438,114002.195312,35175.570312,31255.166016,25825.431641,136640.843750,208262.593750,98075.148438,97693.570312,315891.46875,57980.355469,26148.503906,74688.984375,70468.429688,12721.049805,63415.410156,311286.84375,39355.425781,777863.1250,217513.562500,8567.985352,8546.731445,33525.195312,520774.65625,22935.613281,5042.623047,85089.632812,71819.343750,174409.828125,110728.960938,116357.601562,37467.273438,30295.666016,105823.968750,62730.042969,510416.46875,73769.679688,47705.886719,89578.078125,35050.671875,200614.859375,237845.796875,39111.968750,82685.468750,183003.875000,260605.250000,48305.230469,81538.609375,81516.390625,5852.198730,77938.164062,281433.12500,5947.989746,24132.017578,45721.175781,15031.874023,292158.68750,48100.277344,170243.921875,15630.602539,89778.218750,76080.898438,73201.976562,6977.147461,105784.695312,10095.209961,72299.054688,10360.570312,28424.509766,5674.010254,21034.865234,128097.476562,47471.

## private

In [10]:
# private
xgb_oof_score = []
xgb_pred_dict = {}

for stock_code, df in tqdm(stock_df_dict.items()):
  stock_name = str(stock_list.loc[stock_list['종목코드'] == stock_code]['종목명'].values).strip("['']")
  print(f"\n[종목코드 {stock_code} 종목명 {stock_name} 학습 및 예측]")

  # split train, valid, test (private)
  train = stock_df_dict[stock_code][:-5]
  test = stock_df_dict[stock_code][-5:]

  X_train = train[:-5].drop(columns = ['Date', 'Close'])
  y_train = train[:-5]['Close']
  X_valid = train[-5:].drop(columns = ['Date', 'Close'])
  y_valid = train[-5:]['Close']
  X_test = test.drop(columns = ['Date', 'Close'])  

  # xgboost 
  xgb = XGBRegressor(n_estimators= 10000, learning_rate = 0.05, tree_method = 'gpu_hist', random_state = SEED)
  xgb.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 100, eval_metric = 'mae', verbose = 100)

  xgb_oof = xgb.predict(X_valid)
  xgb_pred = xgb.predict(X_test)
  xgb_pred_dict[stock_code] = xgb_pred

  # validation NMAE score 
  nmae = NMAE(y_valid, xgb_oof)
  print(f"\n{train[-5:]['Date'].min().strftime('%Y-%m-%d')} ~ {train[-5:]['Date'].max().strftime('%Y-%m-%d')} 기간동안의 Private OOF NMAE Score : {nmae:.6f}\n")
  xgb_oof_score.append(nmae)
  print('=' * 120)

print(f'전체 종목에 대한 평균 Private OOF NMAE Score : {np.mean(xgb_oof_score)}')

  0%|          | 0/376 [00:00<?, ?it/s]


[종목코드 000060 종목명 메리츠화재 학습 및 예측]
[07:04:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:30377.5
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:4155.55
[200]	validation_0-mae:3347.88
[300]	validation_0-mae:3420.38
Stopping. Best iteration:
[225]	validation_0-mae:3345.55



  0%|          | 1/376 [00:00<04:29,  1.39it/s]


2021-09-06 ~ 2021-09-10 기간동안의 Private OOF NMAE Score : 10.403836


[종목코드 000080 종목명 하이트진로 학습 및 예측]
[07:04:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:32889
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:722.348
[200]	validation_0-mae:550.783
[300]	validation_0-mae:579.72
Stopping. Best iteration:
[203]	validation_0-mae:542.047



  1%|          | 2/376 [00:01<04:19,  1.44it/s]

...


2021-09-06 ~ 2021-09-10 기간동안의 Private OOF NMAE Score : 1.017691


[종목코드 363280 종목명 티와이홀딩스 학습 및 예측]
[07:08:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:28067
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:304.911
Stopping. Best iteration:
[83]	validation_0-mae:155.917



100%|█████████▉| 375/376 [04:02<00:00,  1.64it/s]


2021-09-06 ~ 2021-09-10 기간동안의 Private OOF NMAE Score : 0.533228


[종목코드 950130 종목명 엑세스바이오 학습 및 예측]
[07:08:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:15678.5
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:1097.52
Stopping. Best iteration:
[41]	validation_0-mae:726.806



100%|██████████| 376/376 [04:02<00:00,  1.55it/s]


2021-09-06 ~ 2021-09-10 기간동안의 Private OOF NMAE Score : 4.266165

전체 종목에 대한 평균 Private OOF NMAE Score : 2.346443313271286


In [11]:
xgb_private_sub = pd.DataFrame({'Day' : pd.date_range('20210913', '20210917', freq = 'B')})

for stock_code, pred in xgb_pred_dict.items():
  stock_pred = pd.DataFrame({stock_code : pred})
  xgb_private_sub = pd.concat([xgb_private_sub, stock_pred], axis = 1)

xgb_private_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,005490,005830,005850,005930,005935,005940,006260,006280,006360,006400,006650,006730,006800,007070,007310,007390,008560,008770,008930,009150,009240,009540,009830,009900,010060,010120,010130,010620,010780,010950,011000,011070,011170,011200,011210,011780,012330,012450,012510,012750,013120,013890,014680,015750,015760,016360,016380,017670,017800,018260,018880,019170,020150,021240,022100,023530,024110,025900,025980,026960,027360,028050,028260,028300,028670,029780,029960,030000,030190,030200,030520,030530,031390,031430,032190,032300,032350,032500,032620,032640,032830,033290,033310,033640,033780,034020,034220,034230,034730,035080,035250,035420,035600,035720,035760,035890,035900,036030,036460,036490,036540,036570,036810,036830,036930,038500,039030,039200,039490,039840,041190,041510,041960,042000,042660,042700,043150,045390,046890,047040,047050,047810,048260,048410,048530,049070,051600,051900,051905,051910,051915,052400,052690,053030,053580,053800,055550,056190,058470,058820,060150,060250,060720,061970,064260,064350,064550,064760,065660,066570,066575,066970,067000,067160,067310,067630,068240,068270,068760,069080,069500,069620,069960,071050,073240,074600,078020,078130,078340,078600,078930,081660,082270,083790,084110,084370,084850,084990,085660,086280,086390,086450,086790,086900,088350,088800,089030,089970,089980,090430,090460,091700,091990,092040,092190,095610,095660,095700,096530,096770,097950,098460,099320,100090,101490,102110,102710,102780,104480,105560,108320,110790,111770,112040,112610,120110,121800,122450,122630,122870,128940,131290,131370,131970,137400,138040,138930,139130,139480,140410,140860,141080,143240,144510,145020,151860,151910,153130,161390,161890,166090,175330,178320,178920,180640,181710,183300,183490,185490,185750,192820,194480,196170,200130,204270,204320,205470,206640,206650,207940,213420,214150,214320,214370,214450,214980,215000,215200,216080,217270,218410,220630,222080,222800,226950,230360,232140,235980,237690,240810,241560,243070,243840,247540,251270,252670,253450,256840,263720,263750,267250,267980,268600,271560,272210,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,298540,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-09-13,28108.398438,35277.746094,63838.695312,168976.609375,93227.023438,16728.806641,51587.750000,85537.054688,104308.554688,730594.0000,54569.691406,228319.734375,36561.972656,60797.542969,20543.339844,2510.279053,25123.105469,5522.830078,427436.81250,54917.531250,21479.007812,35563.035156,9656.978516,8235.834961,30799.968750,149988.375000,49665.160156,70097.515625,51869.496094,274441.50000,291619.78125,86252.968750,116074.187500,35185.925781,33370.394531,26159.509766,147311.359375,216180.218750,100610.671875,100250.906250,349855.34375,60191.921875,27639.490234,75588.085938,71372.625000,13166.859375,69428.226562,334575.62500,46456.437500,781919.2500,231961.343750,10045.241211,8833.448242,34553.601562,511720.46875,22836.480469,5401.936035,89313.226562,73258.398438,185525.156250,120011.710938,112499.945312,40402.476562,29746.869141,135148.812500,67337.218750,508171.68750,73688.718750,50311.476562,93773.851562,35062.812500,214132.359375,245609.984375,36596.253906,89851.164062,191286.171875,278962.12500,50843.949219,94885.617188,82678.890625,5859.177246,80383.109375,292946.84375,6337.238770,23320.345703,49050.597656,14842.762695,303326.28125,49795.332031,170023.15625,16184.663086,67077.382812,74668.812500,77637.03125,7137.370117,110332.015625,10221.028320,75055.914062,10909.801758,29131.849609,5496.170410,23508.685547,129552.015625,62507

# Submission

In [12]:
xgb_sub = pd.concat([xgb_public_sub, xgb_private_sub], ignore_index = True)
xgb_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,005490,005830,005850,005930,005935,005940,006260,006280,006360,006400,006650,006730,006800,007070,007310,007390,008560,008770,008930,009150,009240,009540,009830,009900,010060,010120,010130,010620,010780,010950,011000,011070,011170,011200,011210,011780,012330,012450,012510,012750,013120,013890,014680,015750,015760,016360,016380,017670,017800,018260,018880,019170,020150,021240,022100,023530,024110,025900,025980,026960,027360,028050,028260,028300,028670,029780,029960,030000,030190,030200,030520,030530,031390,031430,032190,032300,032350,032500,032620,032640,032830,033290,033310,033640,033780,034020,034220,034230,034730,035080,035250,035420,035600,035720,035760,035890,035900,036030,036460,036490,036540,036570,036810,036830,036930,038500,039030,039200,039490,039840,041190,041510,041960,042000,042660,042700,043150,045390,046890,047040,047050,047810,048260,048410,048530,049070,051600,051900,051905,051910,051915,052400,052690,053030,053580,053800,055550,056190,058470,058820,060150,060250,060720,061970,064260,064350,064550,064760,065660,066570,066575,066970,067000,067160,067310,067630,068240,068270,068760,069080,069500,069620,069960,071050,073240,074600,078020,078130,078340,078600,078930,081660,082270,083790,084110,084370,084850,084990,085660,086280,086390,086450,086790,086900,088350,088800,089030,089970,089980,090430,090460,091700,091990,092040,092190,095610,095660,095700,096530,096770,097950,098460,099320,100090,101490,102110,102710,102780,104480,105560,108320,110790,111770,112040,112610,120110,121800,122450,122630,122870,128940,131290,131370,131970,137400,138040,138930,139130,139480,140410,140860,141080,143240,144510,145020,151860,151910,153130,161390,161890,166090,175330,178320,178920,180640,181710,183300,183490,185490,185750,192820,194480,196170,200130,204270,204320,205470,206640,206650,207940,213420,214150,214320,214370,214450,214980,215000,215200,216080,217270,218410,220630,222080,222800,226950,230360,232140,235980,237690,240810,241560,243070,243840,247540,251270,252670,253450,256840,263720,263750,267250,267980,268600,271560,272210,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,298540,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-08-30,24925.125000,31336.667969,61190.394531,167265.375000,88352.039062,17018.898438,54904.292969,81078.289062,103442.562500,650774.5625,48399.035156,221944.562500,31986.554688,55878.007812,17921.396484,2374.069824,25877.228516,5391.016602,311403.09375,56400.218750,21530.740234,34567.867188,9902.660156,7983.778320,29158.724609,147513.406250,47799.410156,64871.878906,46388.074219,250994.531250,295004.71875,81688.648438,114002.195312,35175.570312,31255.166016,25825.431641,136640.843750,208262.593750,98075.148438,97693.570312,315891.46875,57980.355469,26148.503906,74688.984375,70468.429688,12721.049805,63415.410156,311286.84375,39355.425781,777863.1250,217513.562500,8567.985352,8546.731445,33525.195312,520774.65625,22935.613281,5042.623047,85089.632812,71819.343750,174409.828125,110728.960938,116357.601562,37467.273438,30295.666016,105823.968750,62730.042969,510416.46875,73769.679688,47705.886719,89578.078125,35050.671875,200614.859375,237845.796875,39111.968750,82685.468750,183003.875000,260605.250000,48305.230469,81538.609375,81516.390625,5852.198730,77938.164062,281433.12500,5947.989746,24132.017578,45721.175781,15031.874023,292158.68750,48100.277344,170243.921875,15630.602539,89778.218750,76080.898438,73201.976562,6977.147461,105784.695312,10095.209961,72299.054688,10360.570312,28424.509766,5674.010254,21034.865234,128097.476562,4747

In [20]:
xgb_sub.to_csv('210912_xgb.csv', index = False)